In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from tensorflow_model_optimization.sparsity import keras as sparsity
from transformers import AutoTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.layers import Input, Dense

ImportError: initialization failed

In [3]:
cwd = os.getcwd()
datapath = os.path.join(cwd, 'dataset')

df = pd.read_excel(os.path.join(datapath, 'all_cleaned.xlsx'))
df = df[['berita', 'label']]

In [4]:
df[df.duplicated()]

,berita,label
19562,quick count indikator data persen jokowi ma ru...,0
21318,NaN,1
21322,NaN,1
21966,NaN,1
23119,NaN,1
23776,NaN,1
23797,NaN,1
23834,raya ulang tahun banyak hadiah sangat indah le...,1
23856,raya ulang tahun banyak hadiah sangat indah le...,1
23870,selamat siang dari tim vaksin gsp kemarin ming...,1


In [5]:
df.drop_duplicates(inplace=True)
df[df.duplicated()]

,berita,label


In [6]:
df.isna().berita.value_counts()

berita
False    27418
True         2
Name: count, dtype: int64

In [7]:
df.dropna(inplace = True)
df.isna().berita.value_counts()

berita
False    27418
Name: count, dtype: int64

In [8]:
df.label.value_counts()

label
0    20944
1     6474
Name: count, dtype: int64

In [9]:
X = df[['berita']]
y = df['label']
rus = RandomUnderSampler(random_state=1, replacement=True)# fit predictor and target variable
X_new, y_new = rus.fit_resample(X,y)
y_new.value_counts()

label
0    6474
1    6474
Name: count, dtype: int64

In [10]:
df_new = X_new
df_new['label'] = y_new
df_new

,berita,label
235,psi sebut banding pecat viani limardi tolak ad...,0
12172,tawar koalisi gerindra pks bilang cari teman l...,0
5192,dpr perintah kpu sepakat honor tugas tps naik ...,0
17290,megawati sebut bicara koalisi capres lihat din...,0
10955,dana milu capai rp triliun jokowi minta detail...,0
...,...,...
27427,raja salman arab saudi bawa orang orang sudah ...,1
27428,hehe selalu senyum lihat tingkah laku pak joko...,1
27429,pak jokowi jadi walikota periode pertama solo ...,1
27430,hari rabu nilai tukar rupiah puruk hingga semp...,1


In [11]:
from tensorflow.keras.callbacks import Callback
            
class LossHistory(Callback):
    def __init__(self):
        super(Callback, self).__init__()
        self.losses = []
        self.val_losses = []

    def on_train_begin(self, logs=None):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs=None):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
#         print(self.losses, self.val_losses)

callback = LossHistory()

In [12]:
df_train, df_test = train_test_split(df_new, test_size=0.3, random_state=42,
                                     stratify=df_new['label'])

df_val, df_test_real = train_test_split(df_test, test_size=0.5, random_state=42,
                                     stratify=df_test['label'])

In [13]:
model_name = 'indobenchmark/indobert-base-p1'
# model_name = 'bert-base-cased'

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

max_len = 70

Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [14]:
X_train = tokenizer(
    text=df_train['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_val = tokenizer(
    text=df_val['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_test = tokenizer(
    text=df_test_real['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [15]:
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids, attention_mask = input_mask)[0]

embeddings = model(input_ids, attention_mask = input_mask)[0] # 0 = last hidden state, 1 = poller_output
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)

y = Dense(1, activation='sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = False

In [20]:
EPOCHS = 3
BATCH_SIZE = 16

end_step = np.ceil(1.0 * len(X_train) / BATCH_SIZE).astype(np.int32) * EPOCHS
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

3


In [22]:
optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
    clipnorm=1.0
)

loss = 'binary_crossentropy'
metric = 'accuracy'

# model.compile(
#     optimizer=optimizer,
#     loss=loss,
#     metrics=metric
# )


In [23]:
new_pruned_model = sparsity.prune_low_magnitude(model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['accuracy'])

ValueError: Subclassed models are not supported currently.

In [25]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 70)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 70)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1244413   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   44         'attention_mask[0][0]']      
                             hidden_state=(None, 70, 76                                     

In [29]:
# from IPython.display import SVG, Image
# from livelossplot.inputs.tf_keras import PlotLossesCallback
# from tensorflow.keras.utils import plot_model

In [30]:
# plot_model(model, to_file='model/model.png', show_shapes=True, show_layer_names=True)
# Image('model.png',width=800, height=2500)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


FileNotFoundError: No such file or directory: 'model.png'

FileNotFoundError: No such file or directory: 'model.png'

<IPython.core.display.Image object>

In [23]:
history = model.fit(
    x = {'input_ids':X_train['input_ids'], 'attention_mask':X_train['attention_mask']},
    y = df_train['label'],
    validation_data = ({'input_ids':X_val['input_ids'], 'attention_mask':X_val['attention_mask']},
                        df_val['label']),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[callback]
)

Epoch 1/3
486/486 [==============================] - 4446s 9s/step - loss: 0.3986 - accuracy: 0.8339 - val_loss: 0.2494 - val_accuracy: 0.9120
Epoch 2/3
486/486 [==============================] - 4588s 9s/step - loss: 0.2448 - accuracy: 0.9062 - val_loss: 0.1921 - val_accuracy: 0.9247
Epoch 3/3
486/486 [==============================] - 5115s 11s/step - loss: 0.2045 - accuracy: 0.9221 - val_loss: 0.1984 - val_accuracy: 0.9255


In [24]:
from sklearn.metrics import classification_report

predicted = model.predict({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']})
y_predicted = [1 if pred >= 0.5 else 0 for pred in predicted]
print(classification_report(df_test_real['label'], y_predicted))

81/81 [==============================] - 1175s 14s/step
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1295
           1       0.97      0.88      0.93      1295

    accuracy                           0.93      2590
   macro avg       0.93      0.93      0.93      2590
weighted avg       0.93      0.93      0.93      2590



In [17]:
df_test['label']

4201     0
16976    0
24437    1
22049    1
25539    1
        ..
25781    1
1215     0
8643     0
12214    0
21544    1
Name: label, Length: 5180, dtype: int64

In [18]:
len(y_predicted)

2590

In [26]:
from datetime import date

today = date.today()
str(today)

'2023-11-15'

In [20]:
'model{}.keras'.format(str(today))

'model2023-11-08.keras'

In [27]:
path_to_dir = 'model_{}'.format(str(today))

In [28]:
model.save('model/model_{}'.format(str(today)))

INFO:tensorflow:Assets written to: model/model_2023-11-15\assets


INFO:tensorflow:Assets written to: model/model_2023-11-15\assets


In [39]:
tf.keras.saving.save_model(
    model, 'model/model2_{}.tf'.format(str(today)), overwrite=True, save_format='tf'
)

INFO:tensorflow:Assets written to: model/model2_2023-11-15.tf\assets


INFO:tensorflow:Assets written to: model/model2_2023-11-15.tf\assets


In [38]:
# model.save('model/model2_{}.keras'.format(str(today)))
loaded_model = tf.keras.saving.load_model('model/model2_{}.keras'.format(str(today)))

TypeError: 'str' object is not callable

In [33]:
model = keras.models.load_model('model/model2_{}.keras'.format(str(today)))

TypeError: 'str' object is not callable

In [22]:
tf.keras.models.save_model(model,'model_{}.keras'.format(str(today)))

C:\Users\LENOVO\anaconda3\lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [46]:
tf.saved_model.save(model, path_to_dir)

INFO:tensorflow:Assets written to: model_3_2023-11-08\assets


INFO:tensorflow:Assets written to: model_3_2023-11-08\assets


In [23]:
model.save('model_2_{}'.format(str(today)))

INFO:tensorflow:Assets written to: model_2_2023-11-08\assets


INFO:tensorflow:Assets written to: model_2_2023-11-08\assets


In [50]:
model.save('model_4_{}.h5'.format(str(today)))

C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [56]:
loaded_model = tf.keras.models.load_model('model_4_{}.h5'.format(str(today)))

ValueError: Unknown layer: 'TFBertModel'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [63]:
import transformers
loaded_model = tf.keras.models.load_model('model_4_2023-11-08.h5', custom_objects={"TFBertModel": transformers.TFBertModel})

MemoryError: Unable to allocate 146. MiB for an array with shape (50000, 768) and data type float32

In [47]:
test_model = tf.saved_model.load('model_2_{}'.format(str(today)))
# test_model2 = tf.keras.models.load_model('model_{}.keras'.format(str(today)))
test_model3 = tf.saved_model.load('model_3_{}'.format(str(today)))

In [49]:
optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
    clipnorm=1.0
)

loss = 'binary_crossentropy'
metric = 'accuracy'

test_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

AttributeError: '_UserObject' object has no attribute 'compile'

In [36]:
test_model([tf.TensorSpec.from_tensor( X_test['input_ids'][:3], name = 'input_ids'), tf.TensorSpec.from_tensor( X_test['attention_mask'][:3], name = 'attention_mask')])

ValueError: Attempt to convert a value (TensorSpec(shape=(3, 70), dtype=tf.int32, name='input_ids')) with an unsupported type (<class 'tensorflow.python.framework.tensor.TensorSpec'>) to a Tensor.

In [33]:
test_model(X_test)

ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * {'attention_mask': <tf.Tensor 'inputs_1:0' shape=(2590, 70) dtype=int32>,
 'input_ids': <tf.Tensor 'inputs:0' shape=(2590, 70) dtype=int32>}
    * False
    * None
  Keyword arguments: {}

 Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (3 total):
    * [TensorSpec(shape=(None, 70), dtype=tf.int32, name='input_ids'),
 TensorSpec(shape=(None, 70), dtype=tf.int32, name='attention_mask')]
    * True
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * [TensorSpec(shape=(None, 70), dtype=tf.int32, name='input_ids'),
 TensorSpec(shape=(None, 70), dtype=tf.int32, name='attention_mask')]
    * False
    * None
  Keyword arguments: {}

In [ ]:
plt.plot(callback.losses)
plt.plot(callback.val_losses)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('step')
plt.legend(['train', 'test'], loc='upper left')
plt.show()